In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
data_train_path="C:\\Users\\somes\\Downloads\\Ball-Classification-main\\Ball-Classification-main\\train"
data_test_path="C:\\Users\\somes\\Downloads\\Ball-Classification-main\\Ball-Classification-main\\test"


batch_size=4
img_size=224
optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001,decay = 1e-4)
loss="categorical_crossentropy"
epochs=50
onnx_path="C:\\Users\\somes\\Downloads\\Ball-Classification-main\\Ball-Classification-main\\model.onnx"


In [3]:
image_generator_test = ImageDataGenerator(rescale=1./255,)    
data_train = image_generator_test.flow_from_directory(batch_size=batch_size,
                                                  directory=data_train_path,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                  class_mode='categorical'
  
                                                  )
data_test = image_generator_test.flow_from_directory(batch_size=batch_size,
                                                  directory=data_test_path,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                  class_mode='categorical'
  
                                                  )

Found 330 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [4]:
print("Train Data",data_train.class_indices)
print("Test Data",data_test.class_indices)



Train Data {'football': 0, 'soccer ball': 1}
Test Data {'football': 0, 'soccer ball': 1}


In [5]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization,concatenate,Conv2DTranspose,Dropout

inputs=Input(shape=(224,224,3),name="input")
    
x = Conv2D(16,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(32,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(128,(3,3), activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(256,(3,3),activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(x)

x=tf.keras.layers.GlobalMaxPool2D()(x)

x=tf.keras.layers.Dense(128,activation="relu")(x)
x=tf.keras.layers.Dense(len(data_train.class_indices),activation="softmax",name="output")(x)

model=tf.keras.Model(inputs=inputs,outputs=x)
model.summary()

model.compile(optimizer =optimizer,loss = loss,  metrics = ["accuracy"])


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        18496 

In [6]:
callbacks=[]
callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss',factor = 0.1,patience = 2,min=1e-6,verbose=1))
callbacks.append(tf.keras.callbacks.EarlyStopping(monitor="loss",patience=4,restore_best_weights=True))

In [7]:
model.fit(data_train,validation_data=data_test,  epochs = 10,callbacks=callbacks)

Epoch 1/10
83/83 [==============================] - 12s 137ms/step - loss: 0.5665 - accuracy: 0.8000 - val_loss: 0.4835 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 2/10
83/83 [==============================] - 10s 126ms/step - loss: 0.2852 - accuracy: 0.8879 - val_loss: 0.3458 - val_accuracy: 0.9000 - lr: 0.0010
Epoch 3/10
83/83 [==============================] - 10s 126ms/step - loss: 0.2324 - accuracy: 0.9152 - val_loss: 0.1311 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/10
83/83 [==============================] - 10s 124ms/step - loss: 0.1267 - accuracy: 0.9576 - val_loss: 0.7235 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 5/10
83/83 [==============================] - 10s 125ms/step - loss: 0.1682 - accuracy: 0.9394 - val_loss: 0.0615 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/10
83/83 [==============================] - 10s 126ms/step - loss: 0.0696 - accuracy: 0.9848 - val_loss: 0.0353 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/10
83/83 [==============================] - 10s 125ms/st

In [8]:
from PIL import Image
import numpy as np

img = Image.open("C:\\Users\\somes\\Downloads\\Ball-Classification-main\\Ball-Classification-main\\test\\football\\football20.png").convert('RGB').resize((img_size,img_size))
img=np.asarray(img)
predicted=model.predict(np.expand_dims(img,axis=0))
predicted
list(data_train.class_indices.keys())[np.argmax(predicted)]

1/1 [==============================] - 0s 83ms/step


'football'

In [9]:
predicted

array([[1., 0.]], dtype=float32)

In [10]:
import os
save_model_path="C:\\Users\\somes\\Downloads\\Ball-Classification-main\\Ball-Classification-main\\saved_model"
if not os.path.isdir(save_model_path):
      os.makedirs(save_model_path)

model.save(save_model_path)

INFO:tensorflow:Assets written to: C:\Users\somes\Downloads\Ball-Classification-main\Ball-Classification-main\saved_model\assets


INFO:tensorflow:Assets written to: C:\Users\somes\Downloads\Ball-Classification-main\Ball-Classification-main\saved_model\assets


In [11]:
#!pip install onnxruntime
#!pip install onnx
#!pip install -U tf2onnx
onnx_path="C:\\Users\\somes\\Downloads\\Ball-Classification-main\\Ball-Classification-main\\model.onnx"


In [12]:
!python -m tf2onnx.convert --saved-model C:/Users/somes/Downloads/Ball-Classification-main/Ball-Classification-main/saved_model --opset 9 --output C:/Users/somes/Downloads/Ball-Classification-main/Ball-Classification-main/model.onnx

2024-01-19 21:35:59.227570: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2024-01-19 21:35:59.227783: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
c:\Users\somes\.conda\envs\unity-classification-p38\lib\runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-01-19 21:36:01.563503: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2024-01-19 21:36:01.564246: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2024-01-19 21:36:01.565170: W t

In [18]:
from PIL import Image
import numpy as np
import onnxruntime as ort

onnx_path="C:\\Users\\somes\\Downloads\\Ball-Classification-main\\Ball-Classification-main\\model.onnx"
img = Image.open("C:\\Users\\somes\\Downloads\\Ball-Classification-main\\Ball-Classification-main\\train\\football\\088.jpg").convert('RGB').resize((224,224))
img=np.expand_dims(np.asarray(img, dtype="float32"),axis=0)
img=img/255

sess_ort = ort.InferenceSession(onnx_path,providers=ort.get_available_providers())

outputs = sess_ort.run(None, {sess_ort.get_inputs()[0].name: img})
print(outputs)

[array([[9.9964440e-01, 3.5559706e-04]], dtype=float32)]
